

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/JohnSnowLabs/spark-nlp-workshop/edit/master/tutorials/streamlit_notebooks/healthcare/NER_PROFESSIONS_ES.ipynb)




# **Detect Professions and Occupations in Spanish text**

## 1. Colab Setup

In [ ]:
import os
import json

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

In [2]:
print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP Version: 3.1.2
SparkNLP-JSL Version: 3.1.2


Install dependencies

In [3]:
%%capture
for k,v in license_keys.items(): 
    %set_env $k=$v

!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jsl_colab_setup.sh
!bash jsl_colab_setup.sh

# Install Spark NLP Display for visualization
!pip install --ignore-installed spark-nlp-display

## 2. Start the Spark session

In [4]:
import json
import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

import sparknlp_jsl
from sparknlp_jsl.annotator import *

spark = sparknlp_jsl.start(license_keys['SECRET'])

spark

## 3. Select the DL model

In [5]:
# If you change the model, re-run all the cells below.
# Applicable models: meddoprof_scielowiki
MODEL_NAME = "meddoprof_scielowiki"

## 4. Some sample examples

In [6]:
# Enter examples to be transformed as strings in this list
text_list = [
    """Paciente varón de 42 años que acude al servicio de Urgencias de su hospital acompañado de las Fuerzas del Orden Público por presentar 
    una actitud hostil y desconfiada hacia su padre, permaneciendo aislado en su habitación desde hace un mes. 
    Se decide ingreso hospitalario en Salud Mental. 
    ANTECEDENTES Antecedentes personales y familiares: niega antecedentes personales y familiares de interés. 
    No reacciones alérgicas medicamentosas. 
    Nunca ha estado en tratamiento psiquiátrico, aunque, al parecer, según los datos aportados por familiares, viene presentando síntomas psicóticos desde hace al menos cinco años. 
    Se trata de un varón divorciado, con una hija de 13 años con la que refiere contacto esporádico. 
    Actualmente en desempleo . 
    Sus familiares verbalizan que desde hace unos cinco años –coincidiendo con la ruptura matrimonial, la pérdida de empleo Fuerzas del Orden Público."""
]

## 5. Define Spark NLP pipeline

In [11]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The model was trained with the bert_portuguese_base_cased embeddings,
# we need to it.
embeddings = WordEmbeddingsModel.pretrained("embeddings_scielowiki_300d", "es", "clinical/models")\
    .setInputCols("document", "token") \
    .setOutputCol("embeddings")

ner_model = MedicalNerModel.pretrained(MODEL_NAME, 'es', 'clinical/models') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

embeddings_scielowiki_300d download started this may take some time.
Approximate size to download 351.2 MB
[OK!]


## 6. Run the pipeline

In [12]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

## 7. Visualize results

In [13]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)